In [1]:
# 데이터로드
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D


In [2]:
# 파일 다운 wget 사용해서 url에서 파일 다운
!wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip

--2024-12-07 05:39:37--  https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip
Resolving sid.erda.dk (sid.erda.dk)... 130.225.104.13
Connecting to sid.erda.dk (sid.erda.dk)|130.225.104.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 276294756 (263M) [application/zip]
Saving to: ‘GTSRB_Final_Training_Images.zip’

GTSRB_Final_Trainin 100%[===================>] 263.50M  24.9MB/s    in 12s     

2024-12-07 05:39:50 (22.4 MB/s) - ‘GTSRB_Final_Training_Images.zip’ saved [276294756/276294756]



In [3]:
import zipfile  # 압축파일 풀기

with zipfile.ZipFile('/content/GTSRB_Final_Training_Images.zip', 'r') as target_file:

  target_file.extractall()

# 대략 43개 디렉토리.. 가지는 이미지가 학습데이터

In [4]:
import os
#프로그램 호환성을 높이기 위해서root_dir 설정해둠
ROOT_DIR = '/content'

DATA_ROOT_DIR = os.path.join(ROOT_DIR, 'GTSRB/Final_Training/Images/')

In [5]:
#데이터 전처리
import os
import cv2
import glob
import numpy as np

image_list = []
label_list = []

image_label_list = os.listdir(DATA_ROOT_DIR)   # 정답리스트

for label_name in image_label_list:  # 정답

  file_path = os.path.join(DATA_ROOT_DIR, label_name)
  img_file_list =  glob.glob(file_path+'/*.ppm')
# ppm 파일 읽어서 리스트에 저장
  for img_file in img_file_list:

    try:
      src_img = cv2.imread(img_file, cv2.IMREAD_COLOR)
      scr_img = cv2.resize(src_img, dsize=(32,32))            # 정답 디렉토리에 있는 OPEN CV 통해 읽기
      src_img = cv2.cvtColor(src_img, cv2.COLOR_BGR2RGB)

      image_list.append(src_img)                #이미지와 정답 리스트에 추가
      label_list.append(float(label_name))    #정답에 해당하는 이미지를 순서대로 리스트에 추가

    except Exception as err:
      print(str(err), img_file)
      continue


In [6]:
import numpy as np
from PIL import Image

# 이미지 크기 통일 (예: 32x32)
target_size = (32, 32)
image_list_resized = [np.array(Image.fromarray(img).resize(target_size)) for img in image_list]
x_train = np.array(image_list_resized).astype('float32')

In [7]:
# 레이블 값이 리스트 형태로 되어 있을 시, 스칼라로 변환해야한다
label_list = [label[0] if isinstance(label, (list, tuple)) else label for label in label_list]
y_train = np.array(label_list).astype('float32')

In [8]:
import numpy as np

s = np.arange(len(x_train))

np.random.shuffle(s)

x_train = x_train[s]
y_train = y_train[s]

In [9]:
SPLIT_RATIO = 0.

SPLIT_NUMS = int(SPLIT_RATIO*len(x_train))

x_test = x_train[0:SPLIT_NUMS:]
y_test = y_train[0:SPLIT_NUMS:]

x_test = x_train[SPLIT_NUMS:]
y_test = y_train[SPLIT_NUMS:]

In [10]:
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

In [11]:
print('x_train.shape =', x_train.shape, ',y_train.shape = ', y_train.shape)
print('x_test.shape =', x_test.shape, ',y_test.shape = ', y_test.shape)

x_train.shape = (39209, 32, 32, 3) ,y_train.shape =  (39209,)
x_test.shape = (39209, 32, 32, 3) ,y_test.shape =  (39209,)


In [13]:
n_classes = len(os.listdir(DATA_ROOT_DIR))

model = Sequential()

model.add(Conv2D(input_shape=(32, 32, 3),kernel_size=(3,3), filters=32, activation='relu'))
model.add(Conv2D(kernel_size=(3,3), filters=64, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 30, 30, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 28, 28, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       3,211,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 43)                  │          11,051 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,241,963 (12.37 MB)

 Trainable params: 3,241,963 (12.37 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
from tensorflow import keras # Import keras from tensorflow

model.compile(loss='sparse_categorical_crossentropy',
              optimizer= keras.optimizers.Adam(), metrics=['accuracy'])

In [22]:
from tensorflow.keras.callbacks import EarlyStopping

earlystopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

hist = model.fit(x_train, y_train, batch_size=32, epochs=30,
                 validation_data=(x_test, y_test), callbacks=[earlystopping])

Epoch 1/30
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 227s 183ms/step - accuracy: 0.5953 - loss: 1.5297 - val_accuracy: 0.9774 - val_loss: 0.1040
Epoch 2/30
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 220s 179ms/step - accuracy: 0.9394 - loss: 0.2038 - val_accuracy: 0.9926 - val_loss: 0.0304
Epoch 3/30
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 219s 179ms/step - accuracy: 0.9674 - loss: 0.1100 - val_accuracy: 0.9967 - val_loss: 0.0150
Epoch 4/30
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 261s 178ms/step - accuracy: 0.9746 - loss: 0.0835 - val_accuracy: 0.9982 - val_loss: 0.0113
Epoch 5/30
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 219s 179ms/step - accuracy: 0.9808 - loss: 0.0652 - val_accuracy: 0.9992 - val_loss: 0.0050
Epoch 6/30
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 262s 179ms/step - accuracy: 0.9831 - loss: 0.0541 - val_accuracy: 0.9995 - val_loss: 0.0044
Epoch 7/30
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 262s 178ms/step - accuracy: 0.9846 - loss: 0.0472 - val_accuracy: 0.9997 - val_loss: 0.0031
Epoch 8/30
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 262s 178ms/step - ac